# EDA - Exploratory Data Analysis

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()
plt.style.use('seaborn-bright')
# print(plt.style.available)

In [ ]:
# df = pd.read_csv('data/train_processed_1.csv', index_col='id')
df = pd.read_csv('data/train_processed_1.csv')
df.head()

## Processing: Income Column

In [ ]:
print('Min income',df['income'].min())
print('Max income',df['income'].max())
print(df['income'].describe())

df['income_log'] = df['income'].apply(np.log).round(2)
print('income log sample data:\n',df['income_log'].head())
print('Min income',df['income_log'].min())
print('Max income',df['income_log'].max())
print(df['income_log'].describe())

In [ ]:
df['income_log'].hist()

In [ ]:
df['income'] = df['income_log']
df.drop('income_log',axis=1,inplace=True)
df.head()

In [ ]:
df['age_in_yrs'].hist()

In [ ]:
df['age_in_yrs'].describe()

In [ ]:
from sklearn import preprocessing
data_scalar = preprocessing.MinMaxScaler()
df['age_in_yrs'] = data_scalar.fit_transform(df[['age_in_yrs']]).flatten()
df['age_in_yrs'].hist()
df.head()

In [ ]:
df['age_in_yrs'].round(3).hist() # minmost  value that retains the plot shape
df['age_in_yrs'] = df['age_in_yrs'].round(3)
df.head()

In [ ]:
df['application_underwriting_score'].hist()

In [ ]:
df['application_underwriting_score'] = data_scalar.fit_transform(df[['application_underwriting_score']]).flatten().round(3)
df['application_underwriting_score'].hist()
df.head()

In [ ]:
df['no_of_premiums_paid'].hist()
df['no_of_premiums_paid'].describe()

In [ ]:
#  tmp = data_scalar.fit_transform(df[['no_of_premiums_paid']]).flatten().round(2)
# pd.Series(tmp).hist()
df['no_of_premiums_paid'] = data_scalar.fit_transform(df[['no_of_premiums_paid']]).flatten().round(2)
df['no_of_premiums_paid'].hist()
df.head()

In [ ]:
df['premium'].hist()

In [ ]:
tmp = data_scalar.fit_transform(df[['premium']]).flatten().round(2)
pd.Series(tmp).hist()

In [ ]:
# tmp2 = np.log(df['premium'])
# pd.Series(tmp2).hist()
df['premium'] = df['premium'].apply(np.log).round(4)
df['premium'].hist()
df.head()

In [ ]:
# df['age_in_yrs'].plot(kind='box')
# df.boxplot(column='age_in_yrs', vert=False)

In [ ]:
# age_bins = [0,40,50,60,80,110]
# df['age_in_yrs'].value_counts(bins=age_bins)

In [ ]:
# df['age_in_yrs'].hist(bins=age_bins, rwidth=.90)

In [ ]:
# df.hist(column='income')
# df['income'].plot(kind='hist', bins=10)
# df['income'].value_counts(bins=10)

In [ ]:
# print( df.no_of_premiums_paid.value_counts().head(5) )
# print( df.no_of_premiums_paid.value_counts().tail(5) )

In [ ]:
'''
# Checking if the values in value_counts() are same even after transformation of keys
res1 = df['count_3-6_months_late'].value_counts().values == df['count_3-6_months_late'].apply(np.log).round(2).value_counts().values
res2 = df['count_6-12_months_late'].value_counts().values == df['count_6-12_months_late'].apply(np.log).round(2).value_counts().values
res3 = df['count_more_than_12_months_late'].value_counts().values == df['count_more_than_12_months_late'].apply(np.log).round(2).value_counts().values
print( np.all(res1) )
print( np.all(res2) )
print( np.all(res3) )
# Because all looks good, lets go ahead and transform the  keys
# Yikes!, this doesn't work because we havev -infinity values :(
'''

In [ ]:
# df['count_3-6_months_late'] = df['count_3-6_months_late'].apply(np.log).round(2)
# df['count_6-12_months_late'] = df['count_6-12_months_late'].apply(np.log).round(2)
# df['count_more_than_12_months_late'] = df['count_more_than_12_months_late'].apply(np.log).round(2)

In [ ]:
# Convert values to Percent in Crosstab, that is to be used sooner..
def toPercent(ser):
    s = ser/ser[-1]*100
    return s

In [ ]:
# df[['residence_area_type','renewal']].groupby(by=['residence_area_type']).aggregate('mean').unstack()
# df[['residence_area_type','renewal']].groupby(by=['residence_area_type']).aggregate(['mean','sum','count'])
pd.crosstab(df['residence_area_type'], df['renewal'], margins=True)

In [ ]:
pd.crosstab(df['residence_area_type'], df['renewal'], margins=True).apply(toPercent, axis=1)

In [ ]:
df['sourcing_channel'].value_counts()

In [ ]:
sns.countplot(df['sourcing_channel'], label='Count')

In [ ]:
pd.crosstab(df['sourcing_channel'], df['renewal'], margins=True)

In [ ]:
# Basic Collerogram
# sns.pairplot(df, kind='reg')
# plot.show()

In [ ]:
# Basic Collerogram
# sns.pairplot(df, kind='scatter', hue='renewal', markers=['o','s'])
# plt.show()

### Define Utility Method categorize(df)

In [ ]:
# Feature Category : sourcing_channel, residence_area_type
from sklearn import preprocessing

def categorize(df):
    '''
    This method takes panda.dataframe having single category column as its data as param.
    It then does apply sklearn.preprocessing.LabelEncoder() on the category column
    and then transforms singular category column into multiple columns using sklearn.preprocessing.OneHotEncoder()
    '''
    prefix = df.columns[0]+'_'
    print('prefix :', prefix)
    # Convert string values to numbers in target column
    le = preprocessing.LabelEncoder()
    df1 = df.apply(le.fit_transform)
    new_cols = [prefix+c for c in le.classes_]
    # Split One Column As Many
    enc = preprocessing.OneHotEncoder() # 1. Instantiate
    arr2d = enc.fit_transform(df1).todense() # 2&3. Fit n Transform returns 2d-array as sparse Matrix
    df2  = pd.DataFrame(arr2d, columns=new_cols) # Convert array-Matrix to Pandas' DataFrame
    return df2    

In [ ]:
# categorize(df[['sourcing_channel']]).head()
tmp_df = categorize(df[['sourcing_channel']])
df2 = df.join(tmp_df)
df2.head()

In [ ]:
tmp_df = categorize(df[['residence_area_type']])
df3 = df2.join(tmp_df)
df3.head()
# Pipeline
# Prediction

In [ ]:
df4 = df3.drop(['sourcing_channel', 'residence_area_type'], axis=1)
df4.head()

In [ ]:
df4.columns

In [ ]:
df4.columns = ['age_in_yrs', 
               'income', 
               'application_underwriting_score', 
               'premium',
               'perc_premium_paid_by_cash_credit', 
               'no_of_premiums_paid',
               'count_3-6_months_late', 'count_6-12_months_late', 'count_more_than_12_months_late', 
               'sourcing_channel_A', 'sourcing_channel_B', 'sourcing_channel_C', 'sourcing_channel_D', 'sourcing_channel_E', 
               'residence_area_type_Rural', 'residence_area_type_Urban',
               'renewal']
df4.head()

In [ ]:
df4.to_csv('data/train_processed_2.csv', index=False)